# Predicción de atrasao con XGBoost

XGBoost es un algoritmo rápido, eficiente y certero basado en Random Forest, o árboles de desisión. Tenemos suficientes observaciones en la data de entrenamiento para utilizar este modelo y relativamente pocas features. Finalmente tenemos features númericas y categoricas que se adaptan a este tipo de modelos.

In [5]:
import pandas as pd
import numpy as np

features_scl = pd.read_csv('../clean_data/synthetic_features.csv', parse_dates=['Fecha-I', 'Fecha-O'])
# Tipo vuelo en variable booleana
features_scl['tipo_b'] = np.where(features_scl['TIPOVUELO']=='N', 1, 0)
periodo_dia_ohe = pd.get_dummies(features_scl['periodo_dia'])
features_scl = features_scl.join(periodo_dia_ohe)
dianom_ohe = pd.get_dummies(features_scl['DIANOM'])
features_scl = features_scl.join(dianom_ohe)
opera_ohe = pd.get_dummies(features_scl['OPERA'])
features_scl = features_scl.join(opera_ohe)

# mean normalization
features_scl['norm_v_last_6'] = (
    (
        features_scl['v_last_6'] -
        features_scl['v_last_6'].mean()
    ) / 
    features_scl['v_last_6'].std()
)

In [6]:
# Columns into model
model_columns = [
    'temporada_alta', 'tipo_b',
    'mañana', 'noche', 'tarde', 
    'Domingo', 'Jueves', 'Lunes', 'Martes',
    'Miercoles', 'Sabado', 'Viernes', 'norm_v_last_6',
    'Aerolineas Argentinas', 'Aeromexico',
    'Air Canada', 'Air France', 'Alitalia', 'American Airlines', 'Austral',
    'Avianca', 'British Airways', 'Copa Air', 'Delta Air', 'Gol Trans',
    'Grupo LATAM', 'Iberia', 'JetSmart SPA', 'K.L.M.', 'Lacsa',
    'Latin American Wings', 'Oceanair Linhas Aereas',
    'Plus Ultra Lineas Aereas', 'Qantas Airways', 'Sky Airline',
    'United Airlines',
]

X = features_scl.loc[:,model_columns]
y = features_scl['atraso_15']

In [12]:
# fit xgboost on an imbalanced classification dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

# define model
model = XGBClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)

print('Mean ROC AUC: %.5f' % mean(scores))

Mean ROC AUC: 0.68004


El modelo XGBoost tiene un roc_auc de 0.680, mejorando un poco frente al regresor lógistico (0.668)
